In [1]:
import time
import requests
import pandas as pd
from  bs4 import BeautifulSoup

In [2]:
start = time.time()

In [3]:
links = pd.read_csv('Data/ml-small/links.csv',dtype={0:'str',1:'str',2:'str'})

imdbId and tmdbId are identifiers used by http://www.imdb.com and https://www.themoviedb.org respectively. So creating vairables containing URL without the identifiers.

In [4]:
tmdb='https://www.themoviedb.org/movie/'
imdb='http://www.imdb.com/title/tt'

Combining the ID with the respective links to get the URLs.

In [5]:
def link_builder(df,new_col,col,link):
    if link=='tmdb':
        df.loc[df[col].notna(),new_col]=df.loc[df[col].notna(),col].apply(lambda x: link+(x))
    else:
        df.loc[df[col].notna(),new_col]=df.loc[df[col].notna(),col].apply(lambda x: link+(x)+'/')

In [6]:
link_builder(links,'imdb_url','imdbId',imdb)
link_builder(links,'tmdb_url','tmdbId',tmdb)

In [7]:
links.isna().sum()

movieId     0
imdbId      0
tmdbId      8
imdb_url    0
tmdb_url    8
dtype: int64

In [8]:
links.head()

,movieId,imdbId,tmdbId,imdb_url,tmdb_url
0,1,0114709,862,http://www.imdb.com/title/tt0114709/,https://www.themoviedb.org/movie/862/
1,2,0113497,8844,http://www.imdb.com/title/tt0113497/,https://www.themoviedb.org/movie/8844/
2,3,0113228,15602,http://www.imdb.com/title/tt0113228/,https://www.themoviedb.org/movie/15602/
3,4,0114885,31357,http://www.imdb.com/title/tt0114885/,https://www.themoviedb.org/movie/31357/
4,5,0113041,11862,http://www.imdb.com/title/tt0113041/,https://www.themoviedb.org/movie/11862/


In [9]:
links.drop(['imdbId','tmdbId'],axis=1)

,movieId,imdb_url,tmdb_url
0,1,http://www.imdb.com/title/tt0114709/,https://www.themoviedb.org/movie/862/
1,2,http://www.imdb.com/title/tt0113497/,https://www.themoviedb.org/movie/8844/
2,3,http://www.imdb.com/title/tt0113228/,https://www.themoviedb.org/movie/15602/
3,4,http://www.imdb.com/title/tt0114885/,https://www.themoviedb.org/movie/31357/
4,5,http://www.imdb.com/title/tt0113041/,https://www.themoviedb.org/movie/11862/
...,...,...,...
9737,193581,http://www.imdb.com/title/tt5476944/,https://www.themoviedb.org/movie/432131/
9738,193583,http://www.imdb.com/title/tt5914996/,https://www.themoviedb.org/movie/445030/
9739,193585,http://www.imdb.com/title/tt6397426/,https://www.themoviedb.org/movie/479308/
9740,193587,http://www.imdb.com/title/tt8391976/,https://www.themoviedb.org/movie/483455/


Using requests and BeautifulSoup4 packages to scrape the plot summaries from the respective websites.

In [11]:
valid_type=(type(BeautifulSoup(requests.get(links['imdb_url'][0]).content,'html.parser').find(class_='inline canwrap').find('span')))
def plot_scapper(url,url_col,valid_type=valid_type):
    if url_col=='tmdb_url':
        element = BeautifulSoup(requests.get(url).content,'html.parser').find(class_='overview')
        if type(element)!=valid_type:
            return None
        else:
            return element.get_text()
    else:
        element = BeautifulSoup(requests.get(url).content,'html.parser').find(class_='inline canwrap').find('span')
        if type(element)!=valid_type:
            return None
        else:
            return element.text

In [12]:
def plot_builder(df,plot_col,url_col):
    df.loc[df[url_col].notna(),plot_col]=df.loc[df[url_col].notna(),url_col].apply(plot_scapper,args=(url_col,))

In [ ]:
plot_builder(links,'imdb_summary','imdb_url')

In [13]:
plot_builder(links,'tmdb_overview','tmdb_url')

KeyboardInterrupt: 

In [ ]:
links.head()

In [ ]:
links.to_csv('url_n_plots.csv',index=False)

In [ ]:
end=time.time()

In [ ]:
print('Run time : {} minutes').format((end-start)/60)